In [10]:
from bs4 import BeautifulSoup
import lxml #is a parser for HTMl
import webscraping
import requests
import os
import re

We shoul generalize these functions... in webscraping.py!

## **[RQ1.1] Get the list of books**

We start to scrape the three pages to extract the URLs for each of them

In [9]:
UrlsFiles = open("urlpages.txt", "w")

url = "https://www.goodreads.com/list/show/1.Best_Books_Ever?page="+str(1)
page = requests.get(url)
soup = BeautifulSoup(page.content, features='lxml')
for a in soup.find_all('a', class_="bookTitle"): #.contents[0].split('\n')[1].strip()
    UrlsFiles.write(a.get('href')+'\n')

#UrlsFiles.truncate(UrlsFiles.tell()-1) see how to remove the '\n'
UrlsFiles.close()

## **RQ[1.2] Crawl books**

In [10]:
headpart = "https://www.goodreads.com"

In [11]:
direct = "htmlpages" #Directory  
parent_dir = "./" #Parent Directory path 
pathAncestor = os.path.join(parent_dir, direct) #Path
os.mkdir(pathAncestor) #create the folder in the path

In [12]:
for i in range(1,101):
    os.makedirs(os.path.join(pathAncestor, 'page ' + str(i)))

In [13]:
UrlsFiles = open("urlpages.txt", "r")

for i, x in enumerate(UrlsFiles,1):
    subdirectory = pathAncestor + "/page " + str(i)
    article_name = "/article_"+str(i)+".html"
    
    complete_path = subdirectory + article_name
    with open(complete_path, "wb") as ip_file:
        link = headpart + x
        page = requests.get(link)
        
        soup = BeautifulSoup(page.text, features='lxml')
        
        ip_file.write(soup.encode('utf-8'))
        ip_file.close()

UrlsFiles.close()

## **RQ[1.3] Parse downloaded pages**

In [151]:
def scrap_book(href):
    r = requests.get(href)
    soup = BeautifulSoup(
        r.text,
        features="lxml"
    )
    
    book_title = soup.find_all('h1')[0].contents[0].replace('\n', '').strip()
    
    author = soup.find_all('span', itemprop='name')[0].contents[0]
    
    print("Description:\n") #show not all but with more link..
    description = ''.join([c for c in soup.find_all('div', id="description")[0].contents[1].text if c != '</br>'])
    print(description)
    
    print("\nDate:")
    date = soup.find_all('div', id="details")[0].contents[3].text.replace('\n', '').strip().split()
    print(date[1]+" "+date[2]+" "+date[3])
    
    ratings = soup.find_all('a', href="#other_reviews")
    rating_count = -1
    rating = -1
    for raiting in ratings:
        if raiting.find_all('meta', itemprop="ratingCount"):
            rating_count = raiting.text.replace('\n', '').strip().split(' ')[0].replace(',', '')
        elif raiting.find_all('meta', itemprop="reviewCount"):
            rating = raiting.text.replace('\n', '').strip().split(' ')[0].replace(',', '')
    
    print(f"\nbook_title: {book_title}\nauthor: {author}\nratings: {rating_count}\nreviews: {rating}")
    
    #Rating Value
    ratingValue = soup.find('span', itemprop="ratingValue")
    print("rating value:", ratingValue.text.strip())
    
    #Number of pages
    numberOfPages = soup.find('span', itemprop="numberOfPages")
    print("numberOfPages:", numberOfPages.text.strip())
    
    #Title series
    series = soup.find_all('a', href= re.compile(r'/series/*'))[0].contents[0].strip()
    print("Title Series:",series)
    
    #Places
    print("\nSetting:")
    for places in soup.find_all('a', href= re.compile(r'/places/*')):
        print(places.text)
        
    #list of characters
    print("\nCharacters:")
    for characters in soup.find_all('a', href= re.compile(r'/characters/*') ):
        print(characters.text)

In [152]:
scrap_book("https://www.goodreads.com/book/show/2767052-the-hunger-games")

Description:

Could you survive on your own in the wild, with every one out to make sure you don't live to see the morning?In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and one girl between the ages 

Date:
September 14th 2008

book_title: The Hunger Games
author: Suzanne Collins
ratings: 6408094
reviews: 172545
rating value: 4.33
numberOfPages: 374 pages
Title Series: (The Hunger Games #1)

Setting:
District 12, Panem
Capitol, Panem
Panem

Characters:
Katniss Everdeen
Peeta Mellark
Cato (Hunger Games)
Primrose Everdeen
Gale Hawthorne
Effie Trinket
Haymitch Abernathy
Cinna
President Coriolanus Snow
Rue
Flavius
Lavinia (Hunger Games)
Marvel
Glimmer
Clove
Foxface
Thresh
Greasy Sae
Madge Undersee
Caesar Flickerman
Claudius Templesmith
Octavia (Hunger Games)
Portia (hunger Games)
